# Covid-19 Challenge: Topic modelling and rule-based matching

COVID-19 Open Research Dataset Challenge (CORD-19)


Within this notebook we want to address the question: 

**"What is known about incubation, transmission, environmental stability and mortality within the COVID-19 literature?"**

In a first iteration, we explored the medical/scientific articles by applying Topic Modelling. We built an LDA and tf-idf model to discover hidden topics and keywords within the medical/scientific articles. We used the gained knowledge to identify common terms within the COVID-19 literature. Next to it, we created an interactive visualization which allow researchers to flexibly explore the topics within the range of available papers.

In a second iteration we used SpaCy's Token Matcher functionality to match sequences of tokens, based on pattern rules. 
We created patterns for the following three items: 

<ul>
<li>Incubation period</li>
<li>Transmission rate and environmental stability</li>
<li>Mortality rate</li>
</ul>

For each of these items we created a dataframe which contains the relevant papers together with the relevant sentences which contain information about incubation periods, transmission rates and mortality rates respectively. In order to help researchers to find these sentences quickly within a given paper, we also included the section title in which the corresponding sencence appears.

Note that we also added extra meta data tags for each of these papers (e.g. if a paper deals with specific risk factor) to provide additional insights. 

# Python libraries

We used following python libraries:

In [1]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

cord19researchchallenge-old
ft-model
pickle



In [2]:
import pandas as pd
import numpy as np
import os
import glob
import spacy
from spacy.matcher import Matcher
import fasttext
import json
import warnings
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.colors as mcolors
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource
from wordcloud import WordCloud

warnings.simplefilter('ignore')
pd.options.mode.chained_assignment = None  # default='warn'

# Data pre-processing
In this chapter we:
 - read the metadata on the CORD-19 papers from the metadata.csv
 - import the body texts and the sections (section title + text) of papers from json files
 - perform some data cleaning
 - detect the paper's language with a fastText model
 - Send the abstract paper text through a SpaCy pipeline (for each paper we add lists of abstract tokens, lemmas, POS tag of tokens, etc. to our dataframe)

## Import the metadata file

In [3]:
data_path = '/kaggle/input/cord19researchchallenge-old/CORD-19-research-challenge/' # path for Kaggle
#data_path = os.getcwd() + '/data/CORD-19-research-challenge/'
meta_df = pd.read_csv(data_path + 'metadata.csv',
                      dtype={'pubmed_id': str, 'Microsoft Academic Paper ID': str, 'doi': str})

## Import the json files

In [4]:
all_json = glob.glob(f'{data_path}/**/*.json', recursive=True)


class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.body_text = []
            self.paragraphs = []
            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            # Paragraps
            for entry in content['body_text']:
                self.paragraphs.append((entry['section'].capitalize().replace('\\', ''), entry['text']))
            self.body_text = '\n'.join(self.body_text)

    def __repr__(self):
        return f'{self.paper_id}: \n\n{self.abstract[:200]}... \n\n{self.body_text[:200]}...'

    
dict_ = {'paper_id': [], 'abstract': [], 'body_text': [], 'authors': [], 'title': [], 'journal': [], 'meta_abstract': [], 'publication_date': [], 'paragraphs': []}

for idx, entry in enumerate(all_json):
    if idx % (len(all_json) // 20) == 0:
        print(f'Processing index: {idx} of {len(all_json)}')
    content = FileReader(entry)
    dict_['paper_id'].append(content.paper_id)
    dict_['body_text'].append(content.body_text)
    dict_['paragraphs'].append(content.paragraphs)

    # get metadata information
    meta_data = meta_df.loc[meta_df['sha'] == content.paper_id]
    dict_['authors'].append(np.nan if meta_data['authors'].empty else meta_data['authors'][0:1].item())
    dict_['title'].append(np.nan if meta_data['title'].empty else meta_data['title'][0:1].item())
    dict_['journal'].append(np.nan if meta_data['journal'].empty else meta_data['journal'][0:1].item())
    dict_['publication_date'].append(np.nan if meta_data['publish_time'].empty else meta_data['publish_time'][0:1].item())
    abstract_text = np.nan if meta_data['abstract'].empty else meta_data['abstract'][0:1].item()
    dict_['abstract'].append(abstract_text if pd.isnull(abstract_text) or abstract_text.partition(' ')[0].lower() != 'abstract' else abstract_text.partition(' ')[2])

df_covid = pd.DataFrame(dict_, columns=['paper_id', 'abstract', 'body_text', 'authors', 'title', 'journal', 'publication_date', 'paragraphs'])

Processing index: 0 of 33375
Processing index: 1668 of 33375
Processing index: 3336 of 33375
Processing index: 5004 of 33375
Processing index: 6672 of 33375
Processing index: 8340 of 33375
Processing index: 10008 of 33375
Processing index: 11676 of 33375
Processing index: 13344 of 33375
Processing index: 15012 of 33375
Processing index: 16680 of 33375
Processing index: 18348 of 33375
Processing index: 20016 of 33375
Processing index: 21684 of 33375
Processing index: 23352 of 33375
Processing index: 25020 of 33375
Processing index: 26688 of 33375
Processing index: 28356 of 33375
Processing index: 30024 of 33375
Processing index: 31692 of 33375
Processing index: 33360 of 33375


## Some initial cleaning steps

- fill the NA-values in the title and abstract with an empty string (necessary for processing with fastText and spaCy)
- concatenate the text paragraphs of the same sections (we will use this to locate our relevant extracts in a paper)

In [5]:
df_covid.abstract.fillna('', inplace=True)
df_covid.title.fillna('', inplace=True)

In [6]:
df_covid.head()

paper_id  \
0  25621281691205eb015383cbac839182b838514f   
1  7db22f7f81977109d493a0edf8ed75562648e839   
2  a137eb51461b4a4ed3980aa5b9cb2f2c1cf0292a   
3  6c3e1a43f0e199876d4bd9ff787e1911fd5cfaa6   
4  2ce201c2ba233a562ee605a9aa12d2719cfa2beb   

                                            abstract  \
0                                                      
1  Scorpine, a small cationic peptide from the ve...   
2  BACKGROUND: The complex interplay between vira...   
3  INTRODUCTION: Understanding the mechanisms und...   
4  BACKGROUND: Human adenovirus type 55 is a re‐e...   

                                           body_text  \
0  Influenza A viruses (IAV) are severe human pat...   
1  The oldest known scorpions lived around 430 mi...   
2  The emergence of Severe Acute Respiratory Synd...   
3  Sjögren's syndrome (SS) is a connective tissue...   
4  Human adenovirus (HAdV) is a common pathogen a...   

                                             authors  \
0                                                NaN   
1  Zhang, Chao; He, Xinlong; Gu, Yaping; Zhou, Hu...   
2  McDermott, Jason E.; Mitchell, Hugh D.; Gralin...   
3  Talotta, Rossella; Sarzi-Puttini, Piercarlo; A...   
4  Yi, Lina; Zou, LiRong; Lu, Jing; Kang, Min; So...   

                                               title  \
0                                                      
1  Recombinant Scorpine Produced Using SUMO Fusio...   
2  The effect of inhibition of PP1 and TNFα signa...   
3  Microbial Agents as Putative Inducers of B Cel...   
4  A cluster of adenovirus type B55 infection in ...   

                          journal publication_date  \
0                             NaN              NaN   
1                        PLoS One       2014-07-28   
2                   BMC Syst Biol       2016-09-23   
3                   J Immunol Res       2019-01-06   
4  Influenza Other Respir Viruses       2017-06-26   

                                          paragraphs  
0  [(, Influenza A viruses (IAV) are severe human...  
1  [(Introduction, The oldest known scorpions liv...  
2  [(Background, The emergence of Severe Acute Re...  
3  [(Introduction, Sjögren's syndrome (SS) is a c...  
4  [(| introduction, Human adenovirus (HAdV) is a...

In [7]:
def merge_section_texts(paragraphs_list):
    """
    Concatenate paragraph texts of the same section
    :param paragraphs_list:
    :return:
    """
    merged_index = 0
    merged_paragraphs = [paragraphs_list[0]]
    for index in range(1, len(paragraphs_list)):
        if merged_paragraphs[merged_index][0] == paragraphs_list[index][0]:
            merged_paragraphs[merged_index] = (merged_paragraphs[merged_index][0], ' '.join(
                [merged_paragraphs[merged_index][1], paragraphs_list[index][1]]))
        else:
            merged_index += 1
            merged_paragraphs.append(paragraphs_list[index])
    return merged_paragraphs


df_covid['sections'] = df_covid.paragraphs.apply(merge_section_texts)
df_covid.drop('paragraphs', axis=1, inplace=True)

## Detection of the language of the papers using fasttext

In [8]:
lang_detect_model = fasttext.load_model('/kaggle/input/ft-model/lid.176.bin') #https://fasttext.cc/docs/en/crawl-vectors.html


def get_language(content):
    """
    Function to detect the language of a string
    :param content: text
    :return: language
    """
    return lang_detect_model.predict(content.replace('\n', ' '))[0][0][-2:]


df_covid['paper_language'] = df_covid['body_text'].apply(get_language)

In [9]:
df_covid.head()

paper_id  \
0  25621281691205eb015383cbac839182b838514f   
1  7db22f7f81977109d493a0edf8ed75562648e839   
2  a137eb51461b4a4ed3980aa5b9cb2f2c1cf0292a   
3  6c3e1a43f0e199876d4bd9ff787e1911fd5cfaa6   
4  2ce201c2ba233a562ee605a9aa12d2719cfa2beb   

                                            abstract  \
0                                                      
1  Scorpine, a small cationic peptide from the ve...   
2  BACKGROUND: The complex interplay between vira...   
3  INTRODUCTION: Understanding the mechanisms und...   
4  BACKGROUND: Human adenovirus type 55 is a re‐e...   

                                           body_text  \
0  Influenza A viruses (IAV) are severe human pat...   
1  The oldest known scorpions lived around 430 mi...   
2  The emergence of Severe Acute Respiratory Synd...   
3  Sjögren's syndrome (SS) is a connective tissue...   
4  Human adenovirus (HAdV) is a common pathogen a...   

                                             authors  \
0                                                NaN   
1  Zhang, Chao; He, Xinlong; Gu, Yaping; Zhou, Hu...   
2  McDermott, Jason E.; Mitchell, Hugh D.; Gralin...   
3  Talotta, Rossella; Sarzi-Puttini, Piercarlo; A...   
4  Yi, Lina; Zou, LiRong; Lu, Jing; Kang, Min; So...   

                                               title  \
0                                                      
1  Recombinant Scorpine Produced Using SUMO Fusio...   
2  The effect of inhibition of PP1 and TNFα signa...   
3  Microbial Agents as Putative Inducers of B Cel...   
4  A cluster of adenovirus type B55 infection in ...   

                          journal publication_date  \
0                             NaN              NaN   
1                        PLoS One       2014-07-28   
2                   BMC Syst Biol       2016-09-23   
3                   J Immunol Res       2019-01-06   
4  Influenza Other Respir Viruses       2017-06-26   

                                            sections paper_language  
0  [(, Influenza A viruses (IAV) are severe human...             en  
1  [(Introduction, The oldest known scorpions liv...             en  
2  [(Background, The emergence of Severe Acute Re...             en  
3  [(Introduction, Sjögren's syndrome (SS) is a c...             en  
4  [(| introduction, Human adenovirus (HAdV) is a...             en

## SpaCy pipeline

First we load the spacy model

In [10]:
nlp = spacy.load('en_core_web_lg') #https://spacy.io/models/en

We send the abstracts through the spaCy pipeline and save the output as a list of Doc objects

In [ ]:
abstracts = list(nlp.pipe(df_covid['abstract'], batch_size=50, n_process=3))

For each paper we add lists of abstract tokens, lemmas, POS tag of tokens, etc. to our dataframe

In [ ]:
df_abstract = df_covid.copy()
df_abstract['token'] = None
df_abstract['lemma'] = None
df_abstract['lemma_lower'] = None
df_abstract['token_pos'] = None
df_abstract['token_tag'] = None
df_abstract['token_dep'] = None
df_abstract['token_entity'] = None

df_abstract['token'] = df_abstract['token'].astype('object')
df_abstract['lemma'] = df_abstract['lemma'].astype('object')
df_abstract['lemma_lower'] = df_abstract['lemma_lower'].astype('object')
df_abstract['token_pos'] = df_abstract['token_pos'].astype('object')
df_abstract['token_tag'] = df_abstract['token_tag'].astype('object')
df_abstract['token_dep'] = df_abstract['token_dep'].astype('object')
df_abstract['token_entity'] = df_abstract['token_entity'].astype('object')


for i in range(len(abstracts)):

    l_token = list()
    l_lemma = list()
    l_lemma_lower = list()
    l_token_pos = list()
    l_token_tag = list()
    l_token_dep = list()
    l_token_entity = list()

    for token in abstracts[i]:
        if not token.is_stop and not token.is_punct and token.is_alpha:
            l_token.append(token.text)
            l_lemma.append(token.lemma_)
            l_lemma_lower.append(token.lemma_.lower())
            l_token_pos.append(token.pos_)
            l_token_tag.append(token.tag_)
            l_token_dep.append(token.dep_)
            l_token_entity.append(token.ent_type_)

    df_abstract.loc[i,'token'] = l_token
    df_abstract.loc[i,'lemma'] = l_lemma
    df_abstract.loc[i,'lemma_lower'] = l_lemma_lower
    df_abstract.loc[i,'token_pos'] = l_token_pos
    df_abstract.loc[i,'token_tag'] = l_token_tag
    df_abstract.loc[i,'token_dep'] = l_token_dep
    df_abstract.loc[i,'token_entity'] = l_token_entity

# Topic modelling (first iteration)

In this chapter, we apply topic modelling on the abstract text of each paper to find related articles. After some extra pre-processing steps, a wordcloud was created in order to get already some first insights. Afterwards we used tf-idf and LDA to uncover keywords and hidden topics within the COVID-19 literature. 

In [ ]:
df_abstract = pd.read_pickle('/kaggle/input/pickle/df_abstract.pickle')

In [ ]:
df_abstract.abstract = df_abstract['abstract'].replace('', np.nan)
df = df_abstract.dropna()

## Exra pre-processing

 Following pre-processing steps were executed: 
 - remove words of certain type (we only include nouns and proper nouns)
 - remove elements with less than 3 characters (in order to exclude typos and meaningless words)

### Remove words of certain type

In [ ]:
for i in range(len(df)):
    for j in range(len(df.lemma.iloc[i])):
        if df.token_pos.iloc[i][j] not in ['NOUN','PROPN']:
            df.lemma.iloc[i][j] = ""    

### remove elements which length < 3

In [ ]:
for i in range(len(df)):
    df.lemma.iloc[i] = [string for string in df.lemma.iloc[i] if not len(string) < 3]

## Word cloud

A word cloud is created to highlight popular or trending terms based on frequency of use and prominence. 

In [ ]:
word_list = list()
for i in range(len(df)):
    for j in range(len(df.lemma.iloc[i])):
        word_list.append(df.lemma.iloc[i][j])

In [ ]:
string_word_list = str(word_list)

string_word_list = string_word_list.replace("'","")

In [ ]:
wordcloud = WordCloud(width = 800, height = 800,background_color ='white').generate(string_word_list)

In [ ]:
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.show() 

## tfidf

We used a tf-idf model to reflect how important a word is to a specific paper within the covid-19 literature. 

### Model

In [ ]:
def get_tf_idf(document_corpus):
# TFIDF -----------------------------------------------------------------------
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), min_df=0.001, max_df=0.85, strip_accents='ascii',
                                       tokenizer=lambda x: x,
                                       preprocessor=lambda x: x)
    tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(document_corpus.lemma).toarray()
    # place tf-idf values in a pandas data frame
    df_tfidf = pd.DataFrame(tfidf_vectorizer_vectors, columns=tfidf_vectorizer.get_feature_names())
    return df_tfidf

In [ ]:
df_tfidf = get_tf_idf(df)

In [ ]:
list(df_tfidf.columns)

### Top 10 key words

Based on the outcome of the tf-idf model, the 10 most frequent terms are derived and listed for every research paper. 

In [ ]:
len(df_tfidf)

In [ ]:
#use a dictionary comprehension to generate the largest_n values in each row of the dataframe. 
#transpose the dataframe and then applied nlargest to each of the columns. 
#use .index.tolist() to extract the desired top_n columns. 
#transposed this result to get the dataframe back into the desired shape.

top_n = 10
df_tfidf_top10 = pd.DataFrame({n: df_tfidf.T[col].nlargest(top_n).index.tolist() 
                  for n, col in enumerate(df_tfidf.T)}).T

In [ ]:
df_tfidf_top10.head(30)

In [ ]:
len(df_tfidf_top10)

## LDA

### Model

An LDA model is trained to discover hidden topics within the COVID-19 literature. This allows users to view each article as a mixture of these topics. Next to it, we clustered all the documents based on the most dominant topic in each document. 
By mapping a specific article into the topic space, we can find related articles. 

In [ ]:
# LDA model -------------------------------------------------------------------

# Creates dictionary, which is a mapping of word IDs to words.
words = corpora.Dictionary(df.lemma)
# Turns each document into a bag of words.
corpus = [words.doc2bow(doc) for doc in df.lemma]

In [ ]:
n_topics = 10
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=words,
                                            num_topics=n_topics,
                                            random_state=2,
                                            update_every=1,
                                            passes=10,
                                            alpha=0.001,
                                            eta=0.001,
                                            per_word_topics=True,
                                            minimum_probability=0)

In [ ]:
print(lda_model.print_topics())

### Evaluation model

The LDA is evaluated based on the following metrics: 

- Perplexity ("commonly used measurement in information theory to evaluate how well a statistical model describes a dataset, with lower perplexity denoting a better probabilistic model.")
- Coherence ("Topic Coherence measures score a single topic by measuring the degree of semantic similarity between high scoring words in the topic.)

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=df.lemma, dictionary=words, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

### pyLDAvis

We used LDAvis, an interactive visualization system, that enables deep inspection of topics and keywords relationships in a LDA analytical model.  
Basically, the visualization has two main pieces. On the left panel, a global view of the discovered topics are displayed. 
Each circle represents a topic and the area of the circles indicate the overall prevalence of the topics among the research papers. 
The closer the topics are located to each other, the more they are related to each other. On the right panel on the other hand, the individual keywords are listed which are most useful for interpreting the currently selected topic. 
So selecting a topic in the left panel reveals the most useful keywords in the right panel for interpreting that selected topic. 
Note that it is also possible to click on individual keywords to get more insights in which topics they occur. 

From this visualisation we derived the following insights. We set the relevance metrix to 0.48 in order to search for relevant topics which are rather unique for a specific topic. 

- **Cluster 1**: describes epidemic diseases and how healthcare and health policy respond to it
- **Cluster 2**: describes reaction of immune system 
- **Cluster 3**: describes patients
- **Cluster 4**: search for antibodies and antigens (also describes a part of the viruses and how they enter our body: cell, virus, peptide, fusion, epitope, entry, membrane, glycoprotein, vector, envelope ...)
- **Cluster 5**: also investigates gene sequences
- **Cluster 6**: Describes other viruses  (influenza, RSV, IBV HRV, HMPV, adenorvirus, rhinovirus, PEDV) 
- **Cluster 7**: Techniques to detect viruses (assay, detection, PCR, method, pcr, test, sequencing, ELISA, microarray)
- **Cluster 8**: describes viruses on animals (What is the reason that this cluster is close to cluster 3?) 
- **Cluster 9**: describes spread of the virus
- **Cluster 10**: describes the viruses 'SARS', 'MERS', 'COV', 'Corona' (on first sight, it seems that this cluster describes the - - diseases that we are interested in)

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

### t-SNE Clustering Chart

Next to pyLDAvis, we also used T-SNE to plot the derived topic clusters in a clustering chart. 

In [ ]:
# Get topic weights and dominant topics ------------
from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook

# Get topic weights
topic_weights = []
for i, row_list in enumerate(lda_model[corpus]):
    topic_weights.append([w for i, w in row_list[0]])

# Array of topic weights    
arr = pd.DataFrame(topic_weights).fillna(0).values

# Keep the well separated points (optional)
arr = arr[np.amax(arr, axis=1) > 0.35]

# Dominant topic number in each doc
topic_num = np.argmax(arr, axis=1)

# tSNE Dimension Reduction
tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')
tsne_lda = tsne_model.fit_transform(arr)

# Plot the Topic Clusters using Bokeh
output_notebook()
n_topics = 4
mycolors = np.array([color for name, color in mcolors.TABLEAU_COLORS.items()])
plot = figure(title="t-SNE Clustering of {} LDA Topics".format(n_topics), 
              plot_width=900, plot_height=700)
plot.scatter(x=tsne_lda[:,0], y=tsne_lda[:,1], color=mycolors[topic_num])

In [ ]:
show(plot)

# Rule-based matching with SpaCy

In [ ]:
# Read the data
#df_covid = pd.read_pickle(os.getcwd() + '/data/processed/df_covid.pickle')
df_covid = pd.read_pickle("df_covid.pickle")

We use spaCy's Token Matcher functionality to match sequences of tokens, based on pattern rules. Compared to using regular expressions on raw text, spaCy’s rule-based matcher engines and components not only let you find the words and phrases you’re looking for – they also give you access to the tokens within the document and their relationships.

- More information about rule-based matching: https://spacy.io/usage/rule-based-matching
- Inspiration: https://www.kaggle.com/cstefanache/nlp-text-mining-disease-behavior

We created patterns for the following three items:
- Incubation period
- Transmission rate and environmental stability
- Mortality rate

For each of these items we searched for relevant papers and passages according to predefined matchers. 
For every item we created a dataframe which contains only papers which deal with one of the items 'incubation period', 'transmission rate' or 'mortality rate'. Next to it, we included the sentences in which relevant information (= matched patterns) were found. So if a certain sentence matches with our predefined patterns, this sentence is included together with the previous and next sentence (to provide some additional context). In order to help researchers to find these sentences quickly within a given paper, we also included the section title in which the corresponding sencence appears. 


## Pre-processing steps

In [11]:
# Initialize the matcher
matcher = Matcher(nlp.vocab)

### Filtering of relevant papers
We search on a synonym list of Covid-19 and SARS-CoV-2 to determine which papers are about the novel corona virus or the disease it causes. In this way we exlude irrelevant papers, since a lot of the available papers are not about COVID-19.

*Inspiration: https://www.kaggle.com/ajrwhite/covid-19-thematic-tagging-with-regular-expressions#Geographies*

In [12]:
# Keep only the English papers
df_covid_en = df_covid[df_covid['paper_language']=='en']
print('Number of English papers in the dataset: ' + str(df_covid_en.shape[0]))

Number of English papers in the dataset: 32655


In [13]:
covid19_synonyms = ['covid',
                    'coronavirus disease 19',
                    'sars cov 2', # Note that search function replaces '-' with ' '
                    '2019 ncov',
                    '2019ncov',
                    r'2019 n cov\b',
                    r'2019n cov\b',
                    'ncov 2019',
                    r'\bn cov 2019',
                    'coronavirus 2019',
                    'wuhan pneumonia',
                    'wuhan virus',
                    'wuhan coronavirus',
                    r'coronavirus 2\b']

# Helper functions
def abstract_title_filter(df, search_string):
    return (df.abstract.str.lower().str.replace('-', ' ').str.contains(search_string, na=False) |
            df.title.str.lower().str.replace('-', ' ').str.contains(search_string, na=False))


def tagger(df, synonym_list, tag_suffix):
    df[f'tag_{tag_suffix}'] = False
    for synonym in synonym_list:
        synonym_filter = abstract_title_filter(df, synonym)
        df.loc[synonym_filter, f'tag_{tag_suffix}'] = True
    return df

In [14]:
# Filter the relevant papers
df_covid_en = tagger(df_covid_en, covid19_synonyms, 'covid19')
df_covid19 = df_covid_en[df_covid_en['tag_covid19']]

In [15]:
print('Number of papers about Covid-19: ' + str(df_covid19.shape[0]))

Number of papers about Covid-19: 1123


### prevent sentence splitting on 'et al.'

We define some additional functions to prevent splitting a sentence in two separate sentences if 'et al.' occurs within it. 

In [16]:
# https://github.com/explosion/spaCy/blob/master/examples/pipeline/custom_sentence_segmentation.py
def prevent_sentence_boundaries(doc):
    for token in doc:
        if not can_be_sentence_start(token):
            token.is_sent_start = False
    return doc


def can_be_sentence_start(token):
    if token.i > 1 and token.nbor(-2).text == 'al':
        return False
    return True

nlp.add_pipe(prevent_sentence_boundaries, before='parser')

### Define additional helper functions for pattern matching

Additional helper functions are defined in order to retrieve a section title from a certain sentence, paragrahps and matches. 

In [17]:
# Helper functions for pattern matching
def get_section_title(span, section_list):
    """
    Function to search in which section of the paper
    the detected pattern match (span) is found.
    """
    section_title = ''
    for section in section_list:
        if span.sent.text.strip() in section[1]:
            section_title = section[0]
    return section_title


def get_paragraph(span, doc):
    l_sents = []
    sentences = list(doc.sents)
    for i in range(len(sentences)):
        if sentences[i].text == span.sent.text:
            if i+1 <= len(sentences)-1 and i-1>=0: 
                l_sents = [sentences[i - 1].text, sentences[i].text, sentences[i + 1].text]
            else:
                l_sents = [sentences[i].text]
    return ' '.join(l_sents)


def get_matches(doc, matcher, matcher_string_id):
    """
    Function to search for all relevant mathes in a document (doc).
    We only match on the requested match patterns 
    (matcher_string_id: list of IDs referring to the matching pattern you want to use)
    """
    matches = []
    raw_matches = matcher(doc)
    matcher_id = []
    for id_ in matcher_string_id:
        matcher_id.append(nlp.vocab.strings[id_])
    for match_id, start, end in raw_matches:
        if end - start < 10 and match_id in matcher_id:
            span = doc[start:end]
            matches.append({'id': match_id, 'span': span})
    return matches


def matched_paragraphs(doc, matcher, matcher_id, section_list):
    matches, l_matched_paragraphs, section_titles = [], [], []
    matches = get_matches(doc, matcher, matcher_id)
    for match in matches:
        section_titles.append(get_section_title(match['span'], section_list))
        l_matched_paragraphs.append(get_paragraph(match['span'], doc))
    matched_paragraphs_tuples = list(zip(section_titles, l_matched_paragraphs))
    unique_matched_paragraphs_tuples = list(dict.fromkeys(matched_paragraphs_tuples).keys())
    return unique_matched_paragraphs_tuples


def create_text_from_paragraph_list(paragraph_tuples):
    tuple_text = []
    for tuple_ in paragraph_tuples:
        tuple_text.append('[{title}] {text}'.format(title=tuple_[0], text=tuple_[1]))
    return ' [...] '.join(tuple_text)

### Send the papers through the spacy pipeline

In [18]:
# docs = list(nlp.pipe(df_covid19['body_text'], batch_size=50, n_process=3))
docs = list(nlp.pipe(df_covid19['body_text'], batch_size=50, n_process=1))

# Add spacy doc objects to our covid-19 df
df_covid19['spacy_doc'] = docs

## Incubation period

### Define patterns


To find all interesting passages about the incubation period we define two patterns to match on. We add them both to our matcher. 

In [19]:
matcher = Matcher(nlp.vocab)
incubation_pattern = [{"LOWER": {'IN': ['incubation', 'latency', 'latent','window']}}, 
                      {"LOWER": 'period'},
                      {'OP':'*'},
                      {'POS': 'NUM'},
                      {'OP':'*'},
                      {"LOWER": {'IN': ['week', 'weeks', 'days','day','months','month']}}]

incubation_pattern2 = [{"LOWER": 'period'},
                       {"LOWER": {'IN': ['of']}},
                       {"LOWER": {'IN': ['incubation', 'latency']}},
                       {'OP':'*'},
                       {'POS': 'NUM'},
                       {'OP':'*'},
                       {"LOWER": {'IN': ['week', 'weeks', 'days','day','month','month']}}]

matcher.add("incubation_matcher", None, incubation_pattern)
matcher.add("incubation_matcher2", None, incubation_pattern2)

In [20]:
incubation_matchers = ['incubation_matcher', 'incubation_matcher2']

### Extract relevant passages from all papers

In [21]:
relevant_extract = []
for index, row in df_covid19.iterrows():
    mp_tuples = matched_paragraphs(row['spacy_doc'], matcher, incubation_matchers, row['sections'])
    relevant_extract.append(create_text_from_paragraph_list(mp_tuples))

df_covid19['incubation_extract'] = relevant_extract
df_covid19['incubation_extract'] = df_covid19['incubation_extract'].apply(lambda x: np.nan if len(x) == 0 else x)

# Create a dataframe with the results
df_incubation = df_covid19.loc[df_covid19['incubation_extract'].notna(), ['paper_id','title','authors', 'incubation_extract']]

In [22]:
print('Number of papers related to Covid-19 that mention the incubation period: ' + str(df_incubation.shape[0]))

Number of papers related to Covid-19 that mention the incubation period: 130


In [23]:
df_incubation.head()

paper_id  \
129   ce358c18aac69fc83c7b2e9a7dca4a43b0f60e2e   
2164  a6ce4ce12c7af1cfb4d71764b963285b687e6b51   
3020  7e8409337e69a72191475029805c6776ad43b60b   
3047  5ba8056230c17ec133169d79aacf61ed7d4b458b   
4662  84398c07143ac22476b0444c7b90cf5e8917c1c1   

                                                  title  \
129   First cases of coronavirus disease 2019 (COVID...   
2164  Assessing the Impact of Reduced Travel on Expo...   
3020  2019-nCoV (Wuhan virus), a novel Coronavirus: ...   
3047     The novel coronavirus outbreak in Wuhan, China   
4662  The value of mitigating epidemic peaks of COVI...   

                                                authors  \
129   Spiteri, Gianfranco; Fielding, James; Diercke,...   
2164  Anzai, Asami; Kobayashi, Tetsuro; Linton, M. N...   
3020  Ralph, R.; Lew, J.; Zeng, T.; Francis, M.; Xue...   
3047                    Zhu, Hengbo; Wei, Li; Niu, Ping   
4662                     Villela, Daniel Antunes Maciel   

                                     incubation_extract  
129   [Epidemiology of first cases in the european r...  
2164  [Statistical model] Assuming the epidemic star...  
3020  [Clinical disease and pathogenesis of 2019-nco...  
3047  [Challenges] It is the first time for the COVI...  
4662  [] Such interventions for COVID-19 would requi...

In [25]:
get_matches(df_covid19.loc[129, 'spacy_doc'], matcher, incubation_matchers)

[{'id': 7457997207366359470,
  'span': incubation period presumed to be up to 14 days}]

### Example: matches found in paper with index 129

### Inspect some results

In [26]:
df_incubation.loc[129, 'incubation_extract']

'[Epidemiology of first cases in the european region] As at 21 February, nine countries had reported cases ( Figure) : Belgium (1), Finland (1), France (12), Germany (16), Italy (3), Russia (2), Spain (2), Sweden (1) and the UK (9 -not included further).\n The place of infection (assessed at national level based on an incubation period presumed to be up to 14 days [11] , travel history and contact with probable or confirmed cases as per the case definition) was reported for 35 cases (missing for three cases), of whom 14 were infected in China (Hubei province: 10 cases; Shandong province: one case; province not reported for three cases). The remaining 21 cases were infected in Europe.'

## Transmission Rate

### Define patterns

To find all interesting passages about the transmission rate we define two patterns to match on. We add both to our matcher. 

In [27]:
matcher = Matcher(nlp.vocab)
transmission_pattern = [{"LOWER": {'IN': ['transmission', 'transmissibility','reproduction']}},
                        {"LOWER":{'IN' : ['number','rate']}},
                       {'OP':'*'},
                       {'POS': 'NUM'}
                       #,{"LOWER": {'IN': ['percent', '%']}}
                       ]

transmission_pattern2 = [{"LOWER": 'reproductive'}, 
                         {"LOWER": 'rate'},
                         {'OP':'*'},
                         {'POS': 'NUM'}
                         #,{"LOWER": {'IN': ['percent', '%']}}
                        ]


transmission_pattern3 = [{"LOWER": 'environmental'}, 
                         {"LOWER": 'stability'},
                         {'OP':'*'},
                         {'POS': 'NUM'}]


matcher.add("transmission_matcher", None, transmission_pattern)
matcher.add("transmission_matcher2", None, transmission_pattern2)
matcher.add("transmission_matcher3", None, transmission_pattern3)

In [28]:
transmission_matchers = ['transmission_matcher', 'transmission_matcher2','transmission_matcher3']

### Extract relevant passages from all papers

In [29]:
# Extract relevant paragraphs

relevant_extract = []
for index, row in df_covid19.iterrows():
    mp_tuples = matched_paragraphs(row['spacy_doc'], matcher, transmission_matchers, row['sections'])
    relevant_extract.append(create_text_from_paragraph_list(mp_tuples))

df_covid19['transmission_extract'] = relevant_extract
df_covid19['transmission_extract'] = df_covid19['transmission_extract'].apply(lambda x: np.nan if len(x) == 0 else x)

# Create a dataframe with the results
df_transmission = df_covid19.loc[df_covid19['transmission_extract'].notna(), ['paper_id','title','authors', 'transmission_extract']]



In [30]:
print('Number of papers related to Covid-19 that mention the transmission rate: ' + str(df_transmission.shape[0]))

Number of papers related to Covid-19 that mention the transmission rate: 132


In [31]:
df_transmission.head()

paper_id  \
1025  bf20dda99538a594eafc258553634fd9195104cb   
2164  a6ce4ce12c7af1cfb4d71764b963285b687e6b51   
2554  28223ad437aa22ac2285bd9dd775e1415a69411a   
2681  2765b43f0c431c63acf11a93ddb513f0332ca22b   
4662  84398c07143ac22476b0444c7b90cf5e8917c1c1   

                                                  title  \
1025  Estimating the Unreported Number of Novel Coro...   
2164  Assessing the Impact of Reduced Travel on Expo...   
2554  Real-time tentative assessment of the epidemio...   
2681  Pattern of early human-to-human transmission o...   
4662  The value of mitigating epidemic peaks of COVI...   

                                                authors  \
1025  Zhao, Shi; Musa, Salihu S.; Lin, Qianying; Ran...   
2164  Anzai, Asami; Kobayashi, Tetsuro; Linton, M. N...   
2554  Wu, Peng; Hao, Xinxin; Lau, Eric H Y; Wong, Je...   
2681                Riou, Julien; Althaus, Christian L.   
4662                     Villela, Daniel Antunes Maciel   

                                   transmission_extract  
1025  [Introduction] deaths in mainland China [4] , ...  
2164  [] Using the estimated parameters and their co...  
2554  [Information on exported cases identified outs...  
2681  [] Since December 2019, China has been experie...  
4662  [] Also, discouraging travel reduces mobility ...

### Example: matches found in paper with index 2164

In [34]:
get_matches(df_covid19.loc[2164, 'spacy_doc'], matcher, transmission_matchers)

[{'id': 403356509576527015, 'span': reproduction number R 0}]

### Inspect some results

In [35]:
df_transmission.loc[2164, 'transmission_extract']

'[] Using the estimated parameters and their covariance matrix, we obtain the expected number of exported cases from Day 58 onwards. Supposing that h(t) is the observed number of cases on day t, the reduced travel volume of exported cases by Day 67 is calculated as:\nWe assumed that the distribution of the number of secondary cases generated by a single primary case follows a negative binomial distribution with the basic reproduction number R 0 , i.e., the average number of secondary cases generated by a single primary case, and the dispersion parameter k. The probability of extinction π defined by the first generating moment [15] is then modeled as:\nR 0 is estimated to range from 1.5 to 3.7, and here we adopt 1.5, 2.2, and 3.7 as plausible values for our calculations [16]'

## Mortality 

### Define patterns

We define the pattern we want to match on and add it to our matcher

In [36]:
# Pattern for which you want to find matches

mortality_pattern = [{"LOWER": {'IN': ['mortality', 'fatality']}},
                     #{'LOWER': {'IN':['rate','rates']}},
                     {'OP': '*'},
                     {'POS': 'NUM'},
                     {"LOWER": {'IN': ['percent', '%']}}]



matcher.add("mortality_matcher", None, mortality_pattern)

### Extract relevant passages from all papers

In [37]:
relevant_extract = []
for index, row in df_covid19.iterrows():
    mp_tuples = matched_paragraphs(row['spacy_doc'], matcher, ['mortality_matcher'], row['sections'])
    relevant_extract.append(create_text_from_paragraph_list(mp_tuples))

df_covid19['mortality_extract'] = relevant_extract
df_covid19['mortality_extract'] = df_covid19['mortality_extract'].apply(lambda x: np.nan if len(x) == 0 else x)

# Create a dataframe with the results
df_mortality = df_covid19.loc[df_covid19['mortality_extract'].notna(), ['paper_id','title','authors', 'mortality_extract']]

In [38]:
print('Number of papers related to Covid-19 that mention the mortality rate: ' + str(df_mortality.shape[0]))

Number of papers related to Covid-19 that mention the mortality rate: 198


In [39]:
df_mortality.head()

paper_id  \
150   469ed0f00c09e2637351c9735c306f27acf3aace   
2554  28223ad437aa22ac2285bd9dd775e1415a69411a   
2815  779c1b5cb3afe3d50219aa2af791014a22eb355a   
3047  5ba8056230c17ec133169d79aacf61ed7d4b458b   
5456  c001abc2e59264f76e44ce7512222c8b007de359   

                                                  title  \
150   First two months of the 2019 Coronavirus Disea...   
2554  Real-time tentative assessment of the epidemio...   
2815  Potential Maternal and Infant Outcomes from (W...   
3047     The novel coronavirus outbreak in Wuhan, China   
5456  Lower mortality of COVID-19 by early recogniti...   

                                                authors  \
150                               Xinguang Chen, Bin Yu   
2554  Wu, Peng; Hao, Xinxin; Lau, Eric H Y; Wong, Je...   
2815              Schwartz, David A.; Graham, Ashley L.   
3047                    Zhu, Hengbo; Wei, Li; Niu, Ping   
5456         Sun, Qin; Qiu, Haibo; Huang, Mao; Yang, Yi   

                                      mortality_extract  
150   [Cumulative number of detected and diagnosed c...  
2554  [] Information on reported cases strongly indi...  
2815  [Sars and pregnancy] A case-control study to d...  
3047  [China's progressive responses] COVID-19 detec...  
5456  [] [5] [6] [7] [8] . Since the outcome of NCP ...

### Example: matches found in paper with index 3265

In [41]:
get_matches(df_covid19.loc[3047, 'spacy_doc'], matcher, ['mortality_matcher'])

[{'id': 4520462362623278102, 'span': fatality rate is much lower (3.37%}]

### Inspect some results

In [45]:
# Inspect some results
df_mortality.loc[3047, 'mortality_extract']

"[China's progressive responses] COVID-19 detection kits have been developed and the test results can be generated within 6 h, which is helpful for early diagnosis, treatment and judgment of the treatment effect. Although the number of patients with COVID-19 infections is large in Wuhan, the fatality rate is much lower (3.37%, by February 19, 2020) compared with that of SARS (11%, 2003). By February 19, 2020, 4895 people have been recovered after treatments and most of them are mild cases.\n"

## Add additional meta data tags

In order to enrich the created dataframes with some additional meta-data, we tagged every paper with relevant themes (e.g. tag_risk_diabetis, tag_risk_mers) using rules based on synonyms and related terms. We apply these rules to the title and abstract of every paper in order to assign a certain tag to it. 

Inspiration: https://www.kaggle.com/ajrwhite/covid-19-thematic-tagging-with-regular-expressions#Geographies

## Keep only desired columns from covid 19 df

In [46]:
columns_to_include = ['paper_id','abstract', 'title','publication_date']
df = df_covid19[columns_to_include]

In [47]:
for i in range(len(df)):
    if df['abstract'].iloc[i] == "":
        df['abstract'].iloc[i] = np.nan
    if df['title'].iloc[i] == "":
        df['title'].iloc[i] = np.nan

## add additional tags

### Severe Acute Respiratory Syndrome (SARS)

In [48]:
sars_synonyms = [r'\bsars\b',
                 'severe acute respiratory syndrome']

In [49]:
df = tagger(df, sars_synonyms, 'sars')

### Middle East Respiratory Syndrome (MERS)

In [50]:
mers_synonyms = [r'\bmers\b',
                 'middle east respiratory syndrome']

In [51]:
df = tagger(df, mers_synonyms, 'mers')

### corona

In [53]:
df = tagger(df, corona_synonyms, 'corona')

In [54]:
corona_synonyms = ['corona', r'\bcov\b']

### Acute Respiratory Distress Syndrome (ARDS)

In [55]:
ards_synonyms = ['acute respiratory distress syndrome',
                 r'\bards\b']

In [56]:
df = tagger(df, ards_synonyms, 'ards')

### Research design

In [57]:
riskfac_synonyms = [
    'risk factor analysis',
    'cross sectional case control',
    'prospective case control',
    'matched case control',
    'medical records review',
    'seroprevalence survey',
    'syndromic surveillance'
]

In [59]:
df = tagger(df, riskfac_synonyms, 'design_riskfac')

### generic risk factors

In [60]:
risk_factor_synonyms = ['risk factor',
                        'risk model',
                        'risk by',
                        'comorbidity',
                        'comorbidities',
                        'coexisting condition',
                        'co existing condition',
                        'clinical characteristics',
                        'clinical features',
                        'demographic characteristics',
                        'demographic features',
                        'behavioural characteristics',
                        'behavioural features',
                        'behavioral characteristics',
                        'behavioral features',
                        'predictive model',
                        'prediction model',
                        'univariate', # implies analysis of risk factors
                        'multivariate', # implies analysis of risk factors
                        'multivariable',
                        'univariable',
                        'odds ratio', # typically mentioned in model report
                        'confidence interval', # typically mentioned in model report
                        'logistic regression',
                        'regression model',
                        'factors predict',
                        'factors which predict',
                        'factors that predict',
                        'factors associated with',
                        'underlying disease',
                        'underlying condition']

In [61]:
df = tagger(df, risk_factor_synonyms, 'generic_risk_factors')

## Demografic risk factors

In [62]:
age_synonyms = ['median age',
                'mean age',
                'average age',
                'elderly',
                r'\baged\b',
                r'\bold',
                'young',
                'teenager',
                'adult',
                'child'
               ]

In [63]:
df = tagger(df, age_synonyms, 'risk_age')

### Gender

In [64]:
sex_synonyms = ['sex',
                'gender',
                r'\bmale\b',
                r'\bfemale\b',
                r'\bmales\b',
                r'\bfemales\b',
                r'\bmen\b',
                r'\bwomen\b'
               ]

In [66]:
df = tagger(df, sex_synonyms, 'risk_gender')

### bodyweight

In [67]:
bodyweight_synonyms = [
    'overweight',
    'over weight',
    'obese',
    'obesity',
    'bodyweight',
    'body weight',
    r'\bbmi\b',
    'body mass',
    'body fat',
    'bodyfat',
    'kilograms',
    r'\bkg\b', # e.g. 70 kg
    r'\dkg\b'  # e.g. 70kg
]

In [68]:
df = tagger(df, bodyweight_synonyms, 'risk_bodyweight')

### Smoking

In [69]:
smoking_synonyms = ['smoking',
                    'smoke',
                    'cigar', # this picks up cigar, cigarette, e-cigarette, etc.
                    'nicotine',
                    'cannabis',
                    'marijuana']

In [70]:
df = tagger(df, smoking_synonyms, 'risk_smoking')

### Diabetes

In [71]:
diabetes_synonyms = [
    'diabet', # picks up diabetes, diabetic, etc.
    'insulin', # any paper mentioning insulin likely to be relevant
    'blood sugar',
    'blood glucose',
    'ketoacidosis',
    'hyperglycemi', # picks up hyperglycemia and hyperglycemic
]

In [72]:
df = tagger(df, diabetes_synonyms, 'risk_diabetes')

### Hypertension

In [73]:
hypertension_synonyms = [
    'hypertension',
    'blood pressure',
    r'\bhbp\b', # HBP = high blood pressure
    r'\bhtn\b' # HTN = hypertension
]

In [74]:
df = tagger(df, hypertension_synonyms, 'risk_hypertension')

### Immunodeficiency

In [75]:
immunodeficiency_synonyms = [
    'immune deficiency',
    'immunodeficiency',
    r'\bhiv\b',
    r'\baids\b'
    'granulocyte deficiency',
    'hypogammaglobulinemia',
    'asplenia',
    'dysfunction of the spleen',
    'spleen dysfunction',
    'complement deficiency',
    'neutropenia',
    'neutropaenia', # alternate spelling
    'cell deficiency' # e.g. T cell deficiency, B cell deficiency
]

In [78]:
df = tagger(df, immunodeficiency_synonyms, 'risk_immunodeficiency')

### Cancer

In [79]:
cancer_synonyms = [
    'cancer',
    'malignant tumour',
    'malignant tumor',
    'melanoma',
    'leukemia',
    'leukaemia',
    'chemotherapy',
    'radiotherapy',
    'radiation therapy',
    'lymphoma',
    'sarcoma',
    'carcinoma',
    'blastoma',
    'oncolog'
]

In [80]:
df = tagger(df, cancer_synonyms, 'risk_cancer')

### Chronic respiratory disease

In [81]:
chronicresp_synonyms = [
    'chronic respiratory disease',
    'asthma',
    'chronic obstructive pulmonary disease',
    r'\bcopd',
    'chronic bronchitis',
    'emphysema'
]

In [82]:
df = tagger(df, chronicresp_synonyms, 'risk_chronic_respiratory_disease')

### Asthma

In [83]:
asthma_synonyms = ['asthma']

In [84]:
df = tagger(df, asthma_synonyms, 'risk_asthma')

### Immunity

In [85]:
immunity_synonyms = [
    'immunity',
    r'\bvaccin',
    'innoculat'
]

In [86]:
df = tagger(df, immunity_synonyms,'immunity_generic')

### Climate

In [87]:
climate_synonyms = [
    'climate',
    'weather',
    'humid',
    'sunlight',
    'air temperature',
    'meteorolog', # picks up meteorology, meteorological, meteorologist
    'climatolog', # as above
    'dry environment',
    'damp environment',
    'moist environment',
    'wet environment',
    'hot environment',
    'cold environment',
    'cool environment'
]

In [88]:
df = tagger(df, climate_synonyms,'climate_generic')

### Add tags based on release date

In [89]:
df['publication_date']= pd.to_datetime(df['publication_date'])

In [90]:
date_sars = pd.to_datetime('2003-04-01')
date_mers = pd.to_datetime('2012-09-01')
date_covid_19 = pd.to_datetime('2019-12-31')

In [92]:
def tagger_date(row):
    date_sars = pd.to_datetime('2003-04-01')
    date_mers = pd.to_datetime('2012-09-01')
    date_covid_19 = pd.to_datetime('2019-12-31')
    
    if row['publication_date'] < date_sars :
      return 'publication date before SARS'
    if ((row['publication_date'] >= date_sars) & (row['publication_date'] < date_mers)):
      return 'publication date after SARS (but before MERS)'
    if ((row['publication_date'] >= date_mers) & (row['publication_date'] < date_covid_19)):
      return 'publication date after MERS (but before COVID 19)'
    if row['publication_date'] >= date_covid_19:
      return 'publication date after COVID_19'

In [93]:
df['tag_publication_date'] = df.apply (lambda row: tagger_date(row), axis=1)

In [94]:
df = df.drop(columns=['abstract', 'title','publication_date'])

## Merge with dataframes

We merge the extra meta data tags with the previously created dataframes (df_covid19, df_mortality, df_incubation and df_transmission). 

In [95]:
df_covid19 = df_covid19.merge(df,on='paper_id',how='left')

In [96]:
df_mortality = df_mortality.merge(df,on='paper_id',how='left')

In [97]:
df_incubation = df_incubation.merge(df,on='paper_id',how='left')

In [98]:
df_transmission = df_transmission.merge(df,on='paper_id',how='left')

## Example dataframes

Below an example of each of the created dataframes is given. 

### Dataframe of papers filtered on covid19 synonyms (and additional meta-data)

In [99]:
df_covid19.head()

paper_id  \
0  11f13e2859eb22b349dbef68fd8124b8ba445aac   
1  ce358c18aac69fc83c7b2e9a7dca4a43b0f60e2e   
2  469ed0f00c09e2637351c9735c306f27acf3aace   
3  123ea6f7fb7c9dbde72ca2a6a5f1c5d0986966ab   
4  0a08fddd9dcee1b1254a05b49113521bbc423ccd   

                                            abstract  \
0  On December 31, 2019, Chinese health officials...   
1  In the WHO European Region, COVID-19 surveilla...   
2  Background Similar to outbreaks of many other ...   
3  The newly identified 2019 novel coronavirus (2...   
4  IMPORTANCE: Health care workers exposed to cor...   

                                           body_text  \
0  cases in the United States. Although these mea...   
1  In the WHO European Region, COVID-19 surveilla...   
2  The epidemic of COVID-19 is caused by a novel ...   
3  Very recently, a novel coronavirus which was t...   
4  Abbreviation: PHQ-9, 9-item Patient Health Que...   

                                             authors  \
0  Patel, Anita; Jernigan, Daniel B.; Abdirizak, ...   
1  Spiteri, Gianfranco; Fielding, James; Diercke,...   
2                              Xinguang Chen, Bin Yu   
3  Tian, Xiaolong; Li, Cheng; Huang, Ailing; Xia,...   
4  Lai, Jianbo; Ma, Simeng; Wang, Ying; Cai, Zhon...   

                                               title  \
0  Initial Public Health Response and Interim Cli...   
1  First cases of coronavirus disease 2019 (COVID...   
2  First two months of the 2019 Coronavirus Disea...   
3  Potent binding of 2019 novel coronavirus spike...   
4  Factors Associated With Mental Health Outcomes...   

                             journal publication_date  \
0          MMWR Morb Mortal Wkly Rep       2020-02-07   
1                      Euro Surveill       2020-03-05   
2  Global Health Research and Policy             2020   
3              Emerg Microbes Infect       2020-02-17   
4                     JAMA Netw Open       2020-03-23   

                                            sections paper_language  \
0  [(, cases in the United States. Although these...             en   
1  [(, In the WHO European Region, COVID-19 surve...             en   
2  [(Introduction, The epidemic of COVID-19 is ca...             en   
3  [(, Very recently, a novel coronavirus which w...             en   
4  [(Etable 1. occupation and geographic data of ...             en   

   tag_covid19  ... tag_risk_smoking tag_risk_diabetes tag_risk_hypertension  \
0         True  ...            False             False                 False   
1         True  ...            False             False                 False   
2         True  ...            False             False                 False   
3         True  ...            False             False                 False   
4         True  ...            False             False                 False   

  tag_risk_immunodeficiency  tag_risk_cancer  \
0                     False            False   
1                     False            False   
2                     False            False   
3                     False            False   
4                     False            False   

   tag_risk_chronic_respiratory_disease  tag_risk_asthma  \
0                                 False            False   
1                                 False            False   
2                                 False            False   
3                                 False            False   
4                                 False            False   

   tag_immunity_generic  tag_climate_generic             tag_publication_date  
0                  True                False  publication date after COVID_19  
1                 False                False  publication date after COVID_19  
2                 False                False  publication date after COVID_19  
3                  True                False  publication date after COVID_19  
4                 False                False  publication date after COVID_19  

[5 rows x 33 columns]

### Dataframe of papers with relevant incubation extracts (and additional meta-data)

In [100]:
df_incubation.head()

paper_id  \
0  ce358c18aac69fc83c7b2e9a7dca4a43b0f60e2e   
1  a6ce4ce12c7af1cfb4d71764b963285b687e6b51   
2  7e8409337e69a72191475029805c6776ad43b60b   
3  5ba8056230c17ec133169d79aacf61ed7d4b458b   
4  84398c07143ac22476b0444c7b90cf5e8917c1c1   

                                               title  \
0  First cases of coronavirus disease 2019 (COVID...   
1  Assessing the Impact of Reduced Travel on Expo...   
2  2019-nCoV (Wuhan virus), a novel Coronavirus: ...   
3     The novel coronavirus outbreak in Wuhan, China   
4  The value of mitigating epidemic peaks of COVI...   

                                             authors  \
0  Spiteri, Gianfranco; Fielding, James; Diercke,...   
1  Anzai, Asami; Kobayashi, Tetsuro; Linton, M. N...   
2  Ralph, R.; Lew, J.; Zeng, T.; Francis, M.; Xue...   
3                    Zhu, Hengbo; Wei, Li; Niu, Ping   
4                     Villela, Daniel Antunes Maciel   

                                  incubation_extract  tag_sars  tag_mers  \
0  [Epidemiology of first cases in the european r...     False     False   
1  [Statistical model] Assuming the epidemic star...     False     False   
2  [Clinical disease and pathogenesis of 2019-nco...      True      True   
3  [Challenges] It is the first time for the COVI...     False     False   
4  [] Such interventions for COVID-19 would requi...     False     False   

   tag_corona  tag_ards  tag_design_riskfac  tag_generic_risk_factors  ...  \
0        True     False               False                     False  ...   
1        True     False               False                      True  ...   
2        True     False               False                     False  ...   
3        True     False               False                     False  ...   
4       False     False               False                     False  ...   

   tag_risk_smoking  tag_risk_diabetes  tag_risk_hypertension  \
0             False              False                  False   
1             False              False                  False   
2             False              False                  False   
3             False              False                  False   
4             False              False                  False   

   tag_risk_immunodeficiency  tag_risk_cancer  \
0                      False            False   
1                      False            False   
2                      False            False   
3                      False            False   
4                      False            False   

   tag_risk_chronic_respiratory_disease  tag_risk_asthma  \
0                                 False            False   
1                                 False            False   
2                                 False            False   
3                                 False            False   
4                                 False            False   

   tag_immunity_generic  tag_climate_generic             tag_publication_date  
0                 False                False  publication date after COVID_19  
1                 False                False  publication date after COVID_19  
2                  True                False  publication date after COVID_19  
3                 False                False  publication date after COVID_19  
4                 False                False  publication date after COVID_19  

[5 rows x 23 columns]

### Dataframe of papers with relevant transmission extracts (and additonal meta-data)

In [101]:
df_transmission.head()

paper_id  \
0  bf20dda99538a594eafc258553634fd9195104cb   
1  a6ce4ce12c7af1cfb4d71764b963285b687e6b51   
2  28223ad437aa22ac2285bd9dd775e1415a69411a   
3  2765b43f0c431c63acf11a93ddb513f0332ca22b   
4  84398c07143ac22476b0444c7b90cf5e8917c1c1   

                                               title  \
0  Estimating the Unreported Number of Novel Coro...   
1  Assessing the Impact of Reduced Travel on Expo...   
2  Real-time tentative assessment of the epidemio...   
3  Pattern of early human-to-human transmission o...   
4  The value of mitigating epidemic peaks of COVI...   

                                             authors  \
0  Zhao, Shi; Musa, Salihu S.; Lin, Qianying; Ran...   
1  Anzai, Asami; Kobayashi, Tetsuro; Linton, M. N...   
2  Wu, Peng; Hao, Xinxin; Lau, Eric H Y; Wong, Je...   
3                Riou, Julien; Althaus, Christian L.   
4                     Villela, Daniel Antunes Maciel   

                                transmission_extract  tag_sars  tag_mers  \
0  [Introduction] deaths in mainland China [4] , ...      True      True   
1  [] Using the estimated parameters and their co...     False     False   
2  [Information on exported cases identified outs...     False     False   
3  [] Since December 2019, China has been experie...      True     False   
4  [] Also, discouraging travel reduces mobility ...     False     False   

   tag_corona  tag_ards  tag_design_riskfac  tag_generic_risk_factors  ...  \
0        True     False               False                     False  ...   
1        True     False               False                      True  ...   
2        True     False               False                      True  ...   
3        True     False               False                     False  ...   
4       False     False               False                     False  ...   

   tag_risk_smoking  tag_risk_diabetes  tag_risk_hypertension  \
0             False              False                  False   
1             False              False                  False   
2             False              False                  False   
3             False              False                  False   
4             False              False                  False   

   tag_risk_immunodeficiency  tag_risk_cancer  \
0                      False            False   
1                      False            False   
2                      False            False   
3                      False            False   
4                      False            False   

   tag_risk_chronic_respiratory_disease  tag_risk_asthma  \
0                                 False            False   
1                                 False            False   
2                                 False            False   
3                                 False            False   
4                                 False            False   

   tag_immunity_generic  tag_climate_generic             tag_publication_date  
0                 False                False  publication date after COVID_19  
1                 False                False  publication date after COVID_19  
2                 False                False  publication date after COVID_19  
3                 False                False  publication date after COVID_19  
4                 False                False  publication date after COVID_19  

[5 rows x 23 columns]

### Dateframe of papers with relevant mortality extracts (and additional meta-data)

In [102]:
df_mortality.head()

paper_id  \
0  469ed0f00c09e2637351c9735c306f27acf3aace   
1  28223ad437aa22ac2285bd9dd775e1415a69411a   
2  779c1b5cb3afe3d50219aa2af791014a22eb355a   
3  5ba8056230c17ec133169d79aacf61ed7d4b458b   
4  c001abc2e59264f76e44ce7512222c8b007de359   

                                               title  \
0  First two months of the 2019 Coronavirus Disea...   
1  Real-time tentative assessment of the epidemio...   
2  Potential Maternal and Infant Outcomes from (W...   
3     The novel coronavirus outbreak in Wuhan, China   
4  Lower mortality of COVID-19 by early recogniti...   

                                             authors  \
0                              Xinguang Chen, Bin Yu   
1  Wu, Peng; Hao, Xinxin; Lau, Eric H Y; Wong, Je...   
2              Schwartz, David A.; Graham, Ashley L.   
3                    Zhu, Hengbo; Wei, Li; Niu, Ping   
4         Sun, Qin; Qiu, Haibo; Huang, Mao; Yang, Yi   

                                   mortality_extract  tag_sars  tag_mers  \
0  [Cumulative number of detected and diagnosed c...     False     False   
1  [] Information on reported cases strongly indi...     False     False   
2  [Sars and pregnancy] A case-control study to d...      True      True   
3  [China's progressive responses] COVID-19 detec...     False     False   
4  [] [5] [6] [7] [8] . Since the outcome of NCP ...     False     False   

   tag_corona  tag_ards  tag_design_riskfac  tag_generic_risk_factors  ...  \
0        True     False               False                     False  ...   
1        True     False               False                      True  ...   
2        True     False               False                     False  ...   
3        True     False               False                     False  ...   
4       False     False               False                     False  ...   

   tag_risk_smoking  tag_risk_diabetes  tag_risk_hypertension  \
0             False              False                  False   
1             False              False                  False   
2             False              False                  False   
3             False              False                  False   
4             False              False                  False   

   tag_risk_immunodeficiency  tag_risk_cancer  \
0                      False            False   
1                      False            False   
2                      False            False   
3                      False            False   
4                      False            False   

   tag_risk_chronic_respiratory_disease  tag_risk_asthma  \
0                                 False            False   
1                                 False            False   
2                                 False            False   
3                                 False            False   
4                                 False            False   

   tag_immunity_generic  tag_climate_generic             tag_publication_date  
0                 False                False  publication date after COVID_19  
1                 False                False  publication date after COVID_19  
2                  True                False  publication date after COVID_19  
3                 False                False  publication date after COVID_19  
4                 False                False  publication date after COVID_19  

[5 rows x 23 columns]